In [1]:
from finrl.train import train
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.meta.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.meta.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

import numpy as np
import pandas as pd

import exchange_calendars as tc
import pytz, os
from stockstats import StockDataFrame as Sdf

# Load env variables
%load_ext dotenv
%dotenv

# install ipywidgets to fix the jupyter notebook warning
# D:\Anaconda3\envs\finrl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#   from .autonotebook import tqdm as notebook_tqdm
# pip install ipywidgets

D:\Anaconda3\envs\finrl\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
candle_time_interval = '15Min'

In [3]:
print(ticker_list)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [4]:
print(INDICATORS)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [5]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim
state_dim

333

In [6]:
# Alpaca
API_KEY = os.environ.get('API_KEY')
API_SECRET = os.environ.get('API_SECRET')
API_BASE_URL = os.environ.get('API_BASE_URL')
data_url = os.environ.get('data_url')
env = StockTradingEnv

## Alpaca API

In [7]:
# import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit

In [8]:
try:
    api = REST(API_KEY, API_SECRET, API_BASE_URL, "v2")
except BaseException:
    raise ValueError("Wrong Account Info!")

In [9]:
TimeFrame(30, TimeFrameUnit.Minute)
"30Min"

'30Min'

In [10]:
# api.get_bars("AAPL", TimeFrame(30, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df
tic="AAPL"
time_interval=TimeFrame(30, TimeFrameUnit.Minute)
NY = "America/New_York"
start_date = pd.Timestamp("2022-07-01", tz=NY)
end_date = pd.Timestamp("2022-07-02", tz=NY)
start_time=(start_date + pd.Timedelta("09:30:00")).isoformat()
end_time=(end_date + pd.Timedelta("15:59:00")).isoformat()

print((start_date + pd.Timedelta("09:30:00")).isoformat())

barset = api.get_bars(tic, time_interval, start=start_time, end=end_time, adjustment='raw').df

2022-07-01T09:30:00-04:00


In [11]:
barset["tic"] = tic
barset = barset.reset_index()

In [12]:
barset

,timestamp,open,high,low,close,volume,trade_count,vwap,tic
0,2022-07-01 13:30:00+00:00,136.0400,138.1300,136.0400,137.7400,9925691,94227,137.032095,AAPL
1,2022-07-01 14:00:00+00:00,137.7500,138.3200,136.0000,136.0900,6514401,61733,136.967888,AAPL
2,2022-07-01 14:30:00+00:00,136.0999,136.9240,135.6600,136.5900,6099492,49910,136.218697,AAPL
3,2022-07-01 15:00:00+00:00,136.5800,137.2500,136.1600,136.6012,4134143,38995,136.747438,AAPL
4,2022-07-01 15:30:00+00:00,136.6000,137.0500,136.2100,136.3500,3119754,30229,136.671412,AAPL
5,2022-07-01 16:00:00+00:00,136.3550,137.1899,136.0718,137.0200,3233266,29110,136.682744,AAPL
6,2022-07-01 16:30:00+00:00,137.0300,137.1200,136.5200,136.5200,2559602,25069,136.778293,AAPL
7,2022-07-01 17:00:00+00:00,136.5050,137.0700,136.4400,137.0500,2557096,24308,136.725663,AAPL
8,2022-07-01 17:30:00+00:00,137.0600,137.7200,137.0200,137.7178,3958510,26593,137.393369,AAPL
9,2022-07-01 18:00:00+00:00,137.7150,138.1000,137.2400,137.8700,2833787,25751,137.690182,AAPL


## Clean Data

In [13]:
DP = DataProcessor(data_source = 'alpaca',
                  API_KEY = API_KEY, 
                  API_SECRET = API_SECRET, 
                  API_BASE_URL = API_BASE_URL
                  )

Alpaca successfully connected


In [14]:
data = DP.download_data(start_date = '2022-07-19', 
                        end_date = '2022-07-20',
                        ticker_list = ticker_list, 
                        time_interval= candle_time_interval)
data

Data before 2022-07-19T15:59:00-04:00 is successfully fetched
Data before 2022-07-20T15:59:00-04:00 is successfully fetched


,timestamp,open,high,low,close,volume,trade_count,vwap,tic
0,2022-07-19 13:30:00,144.4000,145.7500,144.4000,145.0100,140675,2187,145.020214,AXP
1,2022-07-19 13:45:00,145.1200,145.5700,144.5000,145.5300,135443,2378,145.015603,AXP
2,2022-07-19 14:00:00,145.5300,146.4400,145.0200,146.1000,113282,2003,145.653443,AXP
3,2022-07-19 14:15:00,146.0700,147.0400,145.8305,146.8800,174772,3010,146.572968,AXP
4,2022-07-19 14:30:00,146.9091,147.0600,146.4050,146.4700,96269,1692,146.719832,AXP
...,...,...,...,...,...,...,...,...,...
21,2022-07-20 18:45:00,52.4800,52.4854,52.3050,52.4700,162514,1901,52.413078,DOW
22,2022-07-20 19:00:00,52.4700,52.6700,52.4600,52.5300,256917,2328,52.560427,DOW
23,2022-07-20 19:15:00,52.5300,52.6099,52.4500,52.6099,358350,3125,52.533933,DOW
24,2022-07-20 19:30:00,52.6100,52.6500,52.4800,52.5650,336668,3467,52.544714,DOW


In [15]:
data = DP.clean_data(data)
data

The price of the first row for ticker  AAPL  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AMGN  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AXP  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  BA  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CAT  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CRM  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CSCO  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CVX  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DIS  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DOW  is NaN.  It will filled with the first valid price.
The price of the first row for ticker 

,timestamp,open,high,low,close,volume,tic
0,2022-07-19 09:30:00-04:00,147.38,147.3800,147.38,147.380,0.0,AAPL
1,2022-07-19 09:31:00-04:00,147.38,147.3800,147.38,147.380,0.0,AAPL
2,2022-07-19 09:32:00-04:00,147.38,147.3800,147.38,147.380,0.0,AAPL
3,2022-07-19 09:33:00-04:00,147.38,147.3800,147.38,147.380,0.0,AAPL
4,2022-07-19 09:34:00-04:00,147.38,147.3800,147.38,147.380,0.0,AAPL
...,...,...,...,...,...,...,...
24655,2022-07-20 18:45:00,131.00,131.1299,130.86,131.120,174829.0,WMT
24656,2022-07-20 19:00:00,131.11,131.3100,130.91,130.910,275422.0,WMT
24657,2022-07-20 19:15:00,130.91,131.0750,130.75,130.855,205465.0,WMT
24658,2022-07-20 19:30:00,130.85,131.0400,130.80,130.980,276831.0,WMT


In [22]:
data = DP.add_technical_indicator(data, INDICATORS)
data[data['tic'] == 'AXP'].head(50)

Succesfully add technical indicators


,timestamp,open,high,low,close,volume,tic,macd_x,boll_ub_x,boll_lb_x,...,close_30_sma_y,close_60_sma_y,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
1644,2022-07-19 09:30:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,NaN,NaN,...,145.01,145.01,0.0,NaN,NaN,NaN,NaN,NaN,145.01,145.01
1645,2022-07-19 09:31:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1646,2022-07-19 09:32:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1647,2022-07-19 09:33:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1648,2022-07-19 09:34:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1649,2022-07-19 09:35:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1650,2022-07-19 09:36:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1651,2022-07-19 09:37:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1652,2022-07-19 09:38:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01
1653,2022-07-19 09:39:00-04:00,145.01,145.01,145.01,145.01,0.0,AXP,0.0,145.01,145.01,...,145.01,145.01,0.0,145.01,145.01,NaN,NaN,NaN,145.01,145.01


In [17]:
data = DP.add_vix(data)
data

Data before 2022-07-19T15:59:00-04:00 is successfully fetched
Data before 2022-07-20T15:59:00-04:00 is successfully fetched
The price of the first row for ticker  VIXY  is NaN.  It will filled with the first valid price.
Data clean finished!


,timestamp,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,VIXY
0,2022-07-19 09:30:00-04:00,147.3800,147.3800,147.3800,147.3800,0.0,AAPL,0.000000,NaN,NaN,NaN,NaN,NaN,147.380000,147.380000,16.475
1,2022-07-19 09:30:00-04:00,245.2700,245.2700,245.2700,245.2700,0.0,AMGN,0.000000,NaN,NaN,NaN,NaN,NaN,245.270000,245.270000,16.475
2,2022-07-19 09:30:00-04:00,145.0100,145.0100,145.0100,145.0100,0.0,AXP,0.000000,NaN,NaN,NaN,NaN,NaN,145.010000,145.010000,16.475
3,2022-07-19 09:30:00-04:00,151.9000,151.9000,151.9000,151.9000,0.0,BA,0.000000,NaN,NaN,NaN,NaN,NaN,151.900000,151.900000,16.475
4,2022-07-19 09:30:00-04:00,174.8554,174.8554,174.8554,174.8554,0.0,CAT,0.000000,NaN,NaN,NaN,NaN,NaN,174.855400,174.855400,16.475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24655,2022-07-20 19:45:00,520.4700,521.5700,519.1800,519.1800,347225.0,UNH,-1.259985,522.188086,515.608414,43.497144,-24.167519,1.117307,521.045527,524.421882,16.300
24656,2022-07-20 19:45:00,213.4600,213.6400,212.9800,213.3600,437245.0,V,0.179874,214.186655,212.011185,54.130108,47.347664,14.185285,212.987763,212.554473,16.300
24657,2022-07-20 19:45:00,49.3550,49.3700,49.0100,49.0850,4884517.0,VZ,-0.234099,50.326651,49.039029,24.290445,-152.897222,71.048527,49.826527,50.130597,16.300
24658,2022-07-20 19:45:00,38.9150,38.9700,38.8700,38.9100,657995.0,WBA,0.057963,38.999227,38.304003,57.707420,147.660953,26.196917,38.619573,38.645848,16.300


In [18]:
price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix='True')

Successfully transformed into array


In [19]:
price_array

array([[147.38  , 245.27  , 145.01  , ...,  50.35  ,  38.69  , 128.935 ],
       [147.38  , 245.27  , 145.01  , ...,  50.35  ,  38.69  , 128.935 ],
       [147.38  , 245.27  , 145.01  , ...,  50.35  ,  38.69  , 128.935 ],
       ...,
       [153.55  , 246.75  , 148.89  , ...,  49.415 ,  38.87  , 130.855 ],
       [153.2799, 246.955 , 149.0313, ...,  49.355 ,  38.9101, 130.98  ],
       [153.03  , 246.62  , 148.91  , ...,  49.085 ,  38.91  , 130.59  ]])

In [20]:
tech_array

array([[  0.        ,   0.        ,   0.        , ...,   0.        ,
        128.935     , 128.935     ],
       [  0.        , 147.38      , 147.38      , ...,   0.        ,
        128.935     , 128.935     ],
       [  0.        , 147.38      , 147.38      , ...,   0.        ,
        128.935     , 128.935     ],
       ...,
       [  0.50013345, 153.79069591, 152.29258409, ...,  23.80098316,
        130.35629   , 130.06056167],
       [  0.48331213, 153.81376652, 152.30661348, ...,  23.80098316,
        130.40029   , 130.08022833],
       [  0.44469015, 153.78109889, 152.30428111, ...,   5.2679518 ,
        130.43012333, 130.093395  ]])

In [ ]:
turbulence_array

In [ ]:
DP = DataProcessor(data_source = 'alpaca',
                  API_KEY = API_KEY, 
                  API_SECRET = API_SECRET, 
                  API_BASE_URL = API_BASE_URL
                  )

In [ ]:
data = DP.download_data(start_date = '2021-10-01', 
                        end_date = '2021-10-02',
                        ticker_list = ticker_list, 
                        time_interval= candle_time_interval,
                        dataset_path='../../../epocha/dataset/stock')